# Cargar archivos .json.gz a un DataFrame
Selecciona cuántos archivos quieres cargar (por defecto 1).

In [ ]:
import os
import pandas as pd
from ipywidgets import interact, IntSlider

input_dir = 'gharchive_data'

# Listar todos los archivos .json.gz disponibles
all_files = []
for root, _, files in os.walk(input_dir):
    for file_name in files:
        if file_name.endswith('.json.gz'):
            all_files.append(os.path.join(root, file_name))

all_files.sort()  # Opcional: para orden cronológico o alfabético
total_files = len(all_files)
print(f"Archivos encontrados: {total_files}")

In [ ]:
def cargar_archivos(n_archivos=1):
    selected_files = all_files[:n_archivos]
    dataframes = []
    for file_path in selected_files:
        try:
            df = pd.read_json(file_path, lines=True, compression='gzip')
            dataframes.append(df)
            print(f"✅ Cargado: {file_path} ({len(df)} filas)")
        except Exception as e:
            print(f"❌ Error leyendo {file_path}: {e}")
    if dataframes:
        combined_df = pd.concat(dataframes, ignore_index=True)
        print("🔢 DataFrame combinado:")
        display(combined_df.head())
        return combined_df
    else:
        print("No se encontraron archivos .json.gz.")
        return None

In [ ]:
interact(cargar_archivos, n_archivos=IntSlider(min=1, max=total_files, step=1, value=1, description='Archivos'))